<a href="https://colab.research.google.com/github/MudassirABBASSi/Sign-Language-SLR-/blob/main/Sign_Language_(SLR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mudassirabbassi_data1122_path = kagglehub.dataset_download('mudassirabbassi/data1122')

print('Data source import complete.')


# Load Annotations and List Video Files

In [ ]:
import os
import pandas as pd

# Define paths
dataset_path = "/kaggle/input/data1122/New folder/data/data"
annotations_path = os.path.join(dataset_path, "annotations")
videos_path = os.path.join(
    dataset_path, "videos")

# Read annotation files
df_train = pd.read_excel(os.path.join(annotations_path, "train.xlsx"))
df_test = pd.read_excel(os.path.join(annotations_path, "test.xlsx"))
df_dev = pd.read_excel(os.path.join(annotations_path, "dev.xlsx"))

# Display sample data
print("Train Annotations:", df_train.head())
print("Test Annotations:", df_test.head())
print("Dev Annotations:", df_dev.head())

# List sample videos
print("Train Videos:", os.listdir(os.path.join(videos_path, "train"))[:5])
print("Test Videos:", os.listdir(os.path.join(videos_path, "test"))[:5])
print("Dev Videos:", os.listdir(os.path.join(videos_path, "dev"))[:5])


Train Annotations:                                          name    signer  \
0  train/11August_2010_Wednesday_tagesschau-1  Signer08   
1  train/11August_2010_Wednesday_tagesschau-4  Signer08   
2  train/11August_2010_Wednesday_tagesschau-5  Signer08   
3  train/11August_2010_Wednesday_tagesschau-6  Signer08   
4  train/11August_2010_Wednesday_tagesschau-7  Signer08   

                                               gloss  \
0      JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR   
1  ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...   
2  NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...   
3  TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...   
4                       WOLKE LOCH SPEZIELL NORDWEST   

                                                text  
0  und nun die wettervorhersage für morgen donner...  
1  mancherorts regnet es auch länger und ergiebig...  
2  im nordwesten bleibt es heute nacht meist troc...  
3  auch am tag gibt es verbreitet zum teil kräfti...  
4  größere wol

#  Video Preprocessing

In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_rate=5):
    """
    Extracts frames from a video at a specific frame rate and saves them as images.

    :param video_path: Path to the video file.
    :param output_folder: Folder to save extracted frames.
    :param frame_rate: Number of frames to extract per second.
    """
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, fps // frame_rate)

    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Extracted {saved_count} frames from {video_path}")

# Example usage
video_file = os.path.join(videos_path, "train", "05March_2011_Saturday_tagesschau-5450.mp4")
output_folder = "/kaggle/working/extracted_frames/train"

extract_frames(video_file, output_folder)


Extracted 14 frames from /kaggle/input/data1122/New folder/data/data/videos/train/05March_2011_Saturday_tagesschau-5450.mp4


# Feature Extraction Using MediaPipe


In [ ]:
!pip install mediapipe

import mediapipe as mp
import numpy as np
import cv2
import os

# Initialize the holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 47.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.6 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tens

#  Define Function for Feature Extraction
This function processes each frame and extracts:

Pose landmarks (full body)

Hand landmarks (left and right hands)

Face landmarks (expressions)

In [ ]:
def extract_keypoints(image):
    """
    Extracts key points from an image using MediaPipe Holistic model.

    :param image: Input image (frame from video).
    :return: Flattened array of extracted key points.
    """
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)

        # Extract key points
        pose = results.pose_landmarks.landmark if results.pose_landmarks else []
        face = results.face_landmarks.landmark if results.face_landmarks else []
        lh = results.left_hand_landmarks.landmark if results.left_hand_landmarks else []
        rh = results.right_hand_landmarks.landmark if results.right_hand_landmarks else []

        # Convert to NumPy arrays
        pose = np.array([[p.x, p.y, p.z] for p in pose]).flatten() if pose else np.zeros(33*3)
        face = np.array([[f.x, f.y, f.z] for f in face]).flatten() if face else np.zeros(468*3)
        lh = np.array([[l.x, l.y, l.z] for l in lh]).flatten() if lh else np.zeros(21*3)
        rh = np.array([[r.x, r.y, r.z] for r in rh]).flatten() if rh else np.zeros(21*3)

        return np.concatenate([pose, face, lh, rh])


#  Extract Features for Each Frame
Now, loop through all frames and extract key points.

In [ ]:
frame_folder = "/kaggle/working/extracted_frames/train"
features = []

for frame_file in sorted(os.listdir(frame_folder)):
    frame_path = os.path.join(frame_folder, frame_file)
    frame = cv2.imread(frame_path)

    if frame is not None:
        keypoints = extract_keypoints(frame)
        features.append(keypoints)

# Convert to NumPy array
features = np.array(features)
print("Feature shape:", features.shape)  # (num_frames, feature_size)


Feature shape: (14, 1629)


# Prepare Data for Model Training

Now, let’s move forward with preparing the dataset for training.
We will:

1️⃣ Label the extracted features based on the gloss (sign language word/phrase).

2️⃣ Organize the data into training, validation, and test sets.

3️⃣ Train an LSTM model for sequence classification.  
 # Load Annotations (Labels)

In [ ]:
import pandas as pd

# Load train annotations
annotations_path = "/kaggle/input/data1122/New folder/data/data/annotations/train.xlsx"
df_train = pd.read_excel(annotations_path)

# Display first few rows
print(df_train.head())


                                         name    signer  \
0  train/11August_2010_Wednesday_tagesschau-1  Signer08   
1  train/11August_2010_Wednesday_tagesschau-4  Signer08   
2  train/11August_2010_Wednesday_tagesschau-5  Signer08   
3  train/11August_2010_Wednesday_tagesschau-6  Signer08   
4  train/11August_2010_Wednesday_tagesschau-7  Signer08   

                                               gloss  \
0      JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR   
1  ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...   
2  NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...   
3  TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...   
4                       WOLKE LOCH SPEZIELL NORDWEST   

                                                text  
0  und nun die wettervorhersage für morgen donner...  
1  mancherorts regnet es auch länger und ergiebig...  
2  im nordwesten bleibt es heute nacht meist troc...  
3  auch am tag gibt es verbreitet zum teil kräfti...  
4  größere wolkenlücken finden si

# Map Features to Labels

In [ ]:
print(df_train["name"].head(10))  # Print first 10 rows
video_name = "train/05March_2011_Saturday_tagesschau-5450"  # Removed .mp4

matching_rows = df_train[df_train["name"].str.contains(video_name, na=False)]

if not matching_rows.empty:
    gloss_label = matching_rows["gloss"].iloc[0]
    print("Label for this video:", gloss_label)
else:
    print("No matching label found for this video.")
matching_rows = df_train[df_train["name"].str.contains("05March_2011_Saturday_tagesschau-5450", na=False)]
print(df_train[df_train["name"].str.contains("05March", na=False)])


0     train/11August_2010_Wednesday_tagesschau-1
1     train/11August_2010_Wednesday_tagesschau-4
2     train/11August_2010_Wednesday_tagesschau-5
3     train/11August_2010_Wednesday_tagesschau-6
4     train/11August_2010_Wednesday_tagesschau-7
5     train/11August_2010_Wednesday_tagesschau-9
6    train/11August_2010_Wednesday_tagesschau-10
7    train/11August_2010_Wednesday_tagesschau-11
8    train/11August_2010_Wednesday_tagesschau-12
9    train/11August_2010_Wednesday_tagesschau-13
Name: name, dtype: object
Label for this video: MITTWOCH NORDWEST KOMMEN REGEN
                                             name    signer  \
4380  train/05March_2011_Saturday_tagesschau-5438  Signer05   
4381  train/05March_2011_Saturday_tagesschau-5439  Signer05   
4382  train/05March_2011_Saturday_tagesschau-5440  Signer05   
4383  train/05March_2011_Saturday_tagesschau-5441  Signer05   
4384  train/05March_2011_Saturday_tagesschau-5444  Signer05   
4385  train/05March_2011_Saturday_tagesschau-5445  Si

In [ ]:
video_name = "05March_2011_Saturday_tagesschau-5450.mp4"
video_name = "train/05March_2011_Saturday_tagesschau-5450"  # Correct format

matching_rows = df_train[df_train["name"].str.contains(video_name, na=False)]

if not matching_rows.empty:
    gloss_label = matching_rows["gloss"].iloc[0]
    print("Label for this video:", gloss_label)
else:
    print("No matching label found for this video.")


Label for this video: MITTWOCH NORDWEST KOMMEN REGEN


# Plan for Feature Extraction
1️⃣ Extract hand landmarks using MediaPipe Hands

2️⃣ Extract face mesh & lips using MediaPipe Face Mesh

3️⃣ Extract body pose using MediaPipe Pose

4️⃣ Store extracted features in a structured format (CSV/JSON/Pickle)

In [ ]:
pip install opencv-python mediapipe pandas numpy


Note: you may need to restart the kernel to use updated packages.


# checking the availability of the videos for Feature Extraction

In [ ]:
import os

video_folder = "/kaggle/input/data1122/New folder/data/data/videos"

if not os.path.exists(video_folder):
    print(f"❌ Error: Folder does not exist: {video_folder}")
else:
    print(f"✅ Folder exists: {video_folder}")
    print("📂 Listing files in the folder:")

    files = os.listdir(video_folder)
    for file in files:
        print(f"- {file} (Type: {'Folder' if os.path.isdir(os.path.join(video_folder, file)) else 'File'})")

import os

video_folder = "/kaggle/input/data1122/New folder/data/data/videos"

# Search for videos inside all subfolders
video_files = []
for root, dirs, files in os.walk(video_folder):
    for file in files:
        if file.endswith(".mp4"):
            video_files.append(os.path.join(root, file))  # Get full path

print(f"📂 Found {len(video_files)} videos.")
if video_files:
    for video in video_files[:5]:  # Print first 5 videos for verification
        print(f"✅ {video}")
else:
    print("❌ No video files found! Check your dataset.")


✅ Folder exists: /kaggle/input/data1122/New folder/data/data/videos
📂 Listing files in the folder:
- test (Type: Folder)
- train (Type: Folder)
- dev (Type: Folder)
📂 Found 8257 videos.
✅ /kaggle/input/data1122/New folder/data/data/videos/test/24September_2009_Thursday_heute-6091.mp4
✅ /kaggle/input/data1122/New folder/data/data/videos/test/21August_2010_Saturday_tagesschau-8822.mp4
✅ /kaggle/input/data1122/New folder/data/data/videos/test/14December_2010_Tuesday_heute-650.mp4
✅ /kaggle/input/data1122/New folder/data/data/videos/test/05May_2010_Wednesday_tagesschau-3353.mp4
✅ /kaggle/input/data1122/New folder/data/data/videos/test/17January_2011_Monday_tagesschau-7005.mp4


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

# Initialize MediaPipe solutions
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

# Load MediaPipe models
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to extract keypoints
def extract_keypoints(results_hands, results_face, results_pose):
    keypoints = []

    # Extract Hand Landmarks (Left & Right)
    for hand in results_hands.multi_hand_landmarks if results_hands.multi_hand_landmarks else []:
        for landmark in hand.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z])
    keypoints.extend([0] * (21 * 3 * 2 - len(keypoints)))  # Pad if missing

    # Extract Face Mesh Landmarks (468 points)
    if results_face.multi_face_landmarks:
        for landmark in results_face.multi_face_landmarks[0].landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z])
    else:
        keypoints.extend([0] * (468 * 3))  # Pad if missing

    # Extract Pose Landmarks (33 points)
    if results_pose.pose_landmarks:
        for landmark in results_pose.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
    else:
        keypoints.extend([0] * (33 * 4))  # Pad if missing

    return keypoints

# Process all videos in the dataset
video_folder = "/kaggle/input/data1122/New folder/data/data/videos"
output_csv = "extracted_features.csv"
data = []

# Look inside all subfolders
video_files = []
for root, dirs, files in os.walk(video_folder):
    for file in files:
        if file.endswith(".mp4"):
            video_files.append(os.path.join(root, file))  # Get full path

print(f"📂 Found {len(video_files)} videos.")

if not video_files:
    print("❌ No video files found. Check the dataset path.")
else:
    for video_path in video_files:
        cap = cv2.VideoCapture(video_path)
        video_name = os.path.basename(video_path)

        frame_id = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process frame with MediaPipe models
            results_hands = hands.process(rgb_frame)
            results_face = face_mesh.process(rgb_frame)
            results_pose = pose.process(rgb_frame)

            # Extract keypoints
            keypoints = extract_keypoints(results_hands, results_face, results_pose)

            # Store data (video_name, frame_id, features)
            data.append([video_name, frame_id] + keypoints)
            frame_id += 1

        cap.release()

# Save extracted features to CSV
if data:
    columns = ["video_name", "frame_id"] + [f"feature_{i}" for i in range(len(data[0]) - 2)]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"✅ Feature extraction completed! Data saved to {output_csv}")
else:
    print("❌ No features extracted. Check video processing.")


📂 Found 8257 videos.


# Feature Extraction code

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# ✅ Enable OpenCV CUDA if available
if cv2.cuda.getCudaEnabledDeviceCount() > 0:
    print("✅ OpenCV CUDA is enabled!")
    cv2.setUseOptimized(True)
    cv2.cuda.setDevice(0)
else:
    print("⚠️ OpenCV CUDA not available. Using CPU.")

# ✅ Initialize MediaPipe solutions
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

# ✅ Function to extract keypoints (hand, face, pose)
def extract_keypoints(frame):
    global hands, face_mesh, pose  # Use global models to reduce reloading time

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(rgb_frame)
    results_face = face_mesh.process(rgb_frame)
    results_pose = pose.process(rgb_frame)

    keypoints = []

    # Hand Landmarks (21 x 3 per hand)
    if results_hands.multi_hand_landmarks:
        for hand in results_hands.multi_hand_landmarks:
            for landmark in hand.landmark:
                keypoints.extend([landmark.x, landmark.y, landmark.z])
    keypoints.extend([0] * (21 * 3 * 2 - len(keypoints)))  # Pad missing values

    # Face Mesh Landmarks (468 x 3)
    if results_face.multi_face_landmarks:
        for landmark in results_face.multi_face_landmarks[0].landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z])
    else:
        keypoints.extend([0] * (468 * 3))

    # Pose Landmarks (33 x 4)
    if results_pose.pose_landmarks:
        for landmark in results_pose.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
    else:
        keypoints.extend([0] * (33 * 4))

    return keypoints

# ✅ Process a single video
def process_video(video_path):
    video_name = os.path.basename(video_path)
    cap = cv2.VideoCapture(video_path)

    frame_data = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = extract_keypoints(frame)
        frame_data.append([video_name, frame_count] + keypoints)
        frame_count += 1

    cap.release()
    return frame_data

# ✅ Get all video paths
video_root_folder = "/kaggle/input/data1122/New folder/data/data/videos"
video_files = []

for subfolder in ["train", "test", "dev"]:
    subfolder_path = os.path.join(video_root_folder, subfolder)
    for file in os.listdir(subfolder_path):
        if file.endswith(".mp4"):
            video_files.append(os.path.join(subfolder_path, file))

print(f"📂 Found {len(video_files)} videos.")

# ✅ Process videos in parallel using multiple CPU cores
num_workers = max(1, cpu_count() - 2)  # Keep 2 cores free for Kaggle's system
with Pool(num_workers) as pool:
    all_data = list(tqdm(pool.imap(process_video, video_files), total=len(video_files), desc="Extracting Features"))

# ✅ Flatten the extracted data
data = [item for sublist in all_data for item in sublist]

# ✅ Save extracted features to CSV
if data:
    columns = ["video_name", "frame"] + [f"feature_{i}" for i in range(len(data[0]) - 2)]
    df = pd.DataFrame(data, columns=columns)
    output_csv = "extracted_features.csv"
    df.to_csv(output_csv, index=False)
    print(f"✅ Feature extraction completed! Data saved to {output_csv}")
else:
    print("❌ No features extracted! Check the video processing.")


# import os
# import cv2
# import mediapipe as mp
# import numpy as np
# import pandas as pd
# from tqdm import tqdm

# # Initialize MediaPipe solutions
# mp_hands = mp.solutions.hands
# mp_face_mesh = mp.solutions.face_mesh
# mp_pose = mp.solutions.pose

# hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
# face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
# pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# # Function to extract keypoints
# def extract_keypoints(results_hands, results_face, results_pose):
#     keypoints = []

#     # Hand Landmarks
#     if results_hands.multi_hand_landmarks:
#         for hand in results_hands.multi_hand_landmarks:
#             for landmark in hand.landmark:
#                 keypoints.extend([landmark.x, landmark.y, landmark.z])
#     keypoints.extend([0] * (21 * 3 * 2 - len(keypoints)))  # Pad if missing

#     # Face Mesh Landmarks (468 points)
#     if results_face.multi_face_landmarks:
#         for landmark in results_face.multi_face_landmarks[0].landmark:
#             keypoints.extend([landmark.x, landmark.y, landmark.z])
#     else:
#         keypoints.extend([0] * (468 * 3))

#     # Pose Landmarks (33 points)
#     if results_pose.pose_landmarks:
#         for landmark in results_pose.pose_landmarks.landmark:
#             keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
#     else:
#         keypoints.extend([0] * (33 * 4))

#     return keypoints

# # Process all videos in the dataset
# video_root_folder = "/kaggle/input/data1122/New folder/data/data/videos"
# output_csv = "extracted_features.csv"
# data = []

# # Get all video paths from subfolders
# video_files = []
# for subfolder in ["train", "test", "dev"]:
#     subfolder_path = os.path.join(video_root_folder, subfolder)
#     for file in os.listdir(subfolder_path):
#         if file.endswith(".mp4"):
#             video_files.append(os.path.join(subfolder_path, file))

# print(f"📂 Found {len(video_files)} videos.")

# # Process videos
# for video_path in tqdm(video_files, desc="Extracting Features"):
#     video_name = os.path.basename(video_path)
#     cap = cv2.VideoCapture(video_path)

#     frame_count = 0
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Convert to RGB
#         rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#         # Process frame with MediaPipe models
#         results_hands = hands.process(rgb_frame)
#         results_face = face_mesh.process(rgb_frame)
#         results_pose = pose.process(rgb_frame)

#         # Extract keypoints
#         keypoints = extract_keypoints(results_hands, results_face, results_pose)

#         # Store data
#         data.append([video_name, frame_count] + keypoints)
#         frame_count += 1

#     cap.release()

# # Save extracted features to CSV
# if data:
#     columns = ["video_name", "frame"] + [f"feature_{i}" for i in range(len(data[0]) - 2)]
#     df = pd.DataFrame(data, columns=columns)
#     df.to_csv(output_csv, index=False)
#     print(f"✅ Feature extraction completed! Data saved to {output_csv}")
# else:
#     print("❌ No features extracted! Check the video processing.")



ModuleNotFoundError: No module named 'mediapipe'

# checking the csv file of Feature

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/working/extracted_frames/train")
print(df.head())  # Check first few rows
print(df.shape)  # Ensure all frames are captured


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/extracted_frames/train'

# Save Features and Labels

In [ ]:
import pickle

# Save as a dictionary
data = {
    "features": features,
    "label": gloss_label
}

# Save to disk
with open("/kaggle/working/sign_data.pkl", "wb") as f:
    pickle.dump(data, f)

print("Features saved successfully!")


In [ ]:
import pickle

with open("/kaggle/working/sign_data.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))  # Check the type
print(data)  # Inspect the content (if not too large)
